# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,17.52,96,100,7.45,ZA,1712582398
1,1,bethel,41.3712,-73.4140,11.34,51,0,0.45,US,1712582218
2,2,mejit,10.2753,170.8646,26.85,85,99,10.40,MH,1712582398
3,3,hithadhoo,-0.6000,73.0833,29.87,66,91,1.45,MV,1712582399
4,4,invercargill,-46.4000,168.3500,13.34,78,63,1.79,NZ,1712582399


In [3]:
type(city_data_df)

pandas.core.frame.DataFrame

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
# Configure the map
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    alpha = 0.5,
    hover_cols = ["City", "Country"]
)

# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [24]:
# Narrow down cities that fit criteria and drop any results with null values
my_cities = city_data_df[(city_data_df['Humidity']<60) & 
                         (city_data_df['Wind Speed'] < 8) & 
                         (city_data_df['Max Temp'] < 30)]

# Display sample data
len(my_cities)

110

### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_cities[['City','Country', 'Lng', 'Lat', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lng,Lat,Humidity,Hotel Name
1,bethel,US,-73.4140,41.3712,51,
6,awjilah,LY,21.2869,29.1081,25,
11,kristiansund,NO,7.7320,63.1115,57,
14,tarinkot,AF,65.8733,32.6268,17,
38,shache,CN,77.2406,38.4167,19,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{row['Lng']},{row['Lat']},{radius}"
    params["bias"] = f"proximity:{row['Lng']},{row['Lat']}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Add the category to the parameters
    params["categories"] = "accommodation.hotel"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
awjilah - nearest hotel: No hotel found
kristiansund - nearest hotel: Comfort Hotel Fosna
tarinkot - nearest hotel: No hotel found
shache - nearest hotel: 莎車新盛賓舘
mogocha - nearest hotel: 7Ka
port lincoln - nearest hotel: Boston Hotel
margaret river - nearest hotel: Margaret River Hotel
parral - nearest hotel: Hotel Adriana
trashigang - nearest hotel: Druk Deothjung Hotel
starodub - nearest hotel: Гостиница "Заря"
malpica - nearest hotel: Pensión Panchito
crane - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
santa elena de uairen - nearest hotel: Posada Villa Fairmont
ankang - nearest hotel: 金友商务酒店
jamestown - nearest hotel: DoubleTree Jamestown
morshyn - nearest hotel: Борис
revolucion mexicana - nearest hotel: No hotel found
quartu sant'elena - nearest hotel: Hotel Italia
klyuchi - nearest hotel: No hotel found
elassona - nearest hotel: Melastron
korocha - nearest hotel: No hotel found
rosoman - n

,City,Country,Lng,Lat,Humidity,Hotel Name
1,bethel,US,-73.4140,41.3712,51,Hampton Inn Danbury
6,awjilah,LY,21.2869,29.1081,25,No hotel found
11,kristiansund,NO,7.7320,63.1115,57,Comfort Hotel Fosna
14,tarinkot,AF,65.8733,32.6268,17,No hotel found
38,shache,CN,77.2406,38.4167,19,莎車新盛賓舘
...,...,...,...,...,...,...
565,warrnambool,AU,142.4833,-38.3833,58,Central Court
570,westport,US,-73.3579,41.1415,49,Norwalk Inn and Conference Center
580,dali,CN,100.1833,25.7000,46,风花雪月大酒店
581,freeport,US,-73.5832,40.6576,48,The Freeport Inn and Marina


In [28]:
# removing records with no hotels found
hotel_df = hotel_df[hotel_df['Hotel Name']!="No hotel found"]
hotel_df

,City,Country,Lng,Lat,Humidity,Hotel Name
1,bethel,US,-73.4140,41.3712,51,Hampton Inn Danbury
11,kristiansund,NO,7.7320,63.1115,57,Comfort Hotel Fosna
38,shache,CN,77.2406,38.4167,19,莎車新盛賓舘
44,mogocha,RU,119.7667,53.7333,59,7Ka
46,port lincoln,AU,135.8667,-34.7333,54,Boston Hotel
...,...,...,...,...,...,...
564,tsumeb,NaN,17.7167,-19.2333,31,Minen Hotel
565,warrnambool,AU,142.4833,-38.3833,58,Central Court
570,westport,US,-73.3579,41.1415,49,Norwalk Inn and Conference Center
580,dali,CN,100.1833,25.7000,46,风花雪月大酒店


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
# Configure the map
map_plot_3 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 600,
    size = "Humidity",
    scale = 1.2,
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot
map_plot_3

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)